<a href="https://colab.research.google.com/github/Munfred/scdefg/blob/main/scdefg1025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!git clone https://github.com/Munfred/scdefg.git

Cloning into 'scdefg'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.


In [13]:
!pip install Flask-Misaka
!pip install Flask-Markdown
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

     |████████████████████████████████| 133kB 2.8MB/s 
  Created wheel for misaka: filename=misaka-2.1.1-cp36-cp36m-linux_x86_64.whl size=129581 sha256=cfccfcc90b50aadc94c6686a6756bb803ac0aa490e1bcd9197bd0453b66d9508
  Stored in directory: /root/.cache/pip/wheels/19/f9/c6/816ca0b45533c2d046c44dc4cbbdf3ad049f62d60f79117683
Successfully built misaka


In [64]:
from flask import Flask,  jsonify, request, render_template, Blueprint, send_file, redirect, url_for
import logging
import pandas as pd
import sys
import json
import time
from io import StringIO
import urllib
from flaskext.markdown import Markdown
from flask_misaka import Misaka
from flask_ngrok import run_with_ngrok
from google.colab import files


In [92]:
logging.basicConfig(level=logging.INFO)
logger=logging.getLogger(__name__)
logger.info('Starting wormcells-de...')

flask_app = Flask(__name__, template_folder='./scdefg/templates', static_folder='./scdefg/static')
flask_app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

run_with_ngrok(flask_app)
Misaka(flask_app, math_explicit = True)


tables = Blueprint('tables', __name__, url_prefix='/tables')

# df with the number of cells of each label in each dataset
df = pd.read_csv(flask_app.open_resource('./scdefg/df.csv'))

# to render the table titles better we replace underscores with spaces,
# use non breaking hyphens (&#8209;) and say batch1 instead of just 1
df_nice_names = df.copy()
df_nice_names.columns = df_nice_names.columns.str.replace('_',' ')
df_nice_names.columns = df_nice_names.columns.str.replace('cho-1 1','cho-1 batch1')
df_nice_names.columns = df_nice_names.columns.str.replace('cho-1 2','cho-1 batch2')
df_nice_names.columns = df_nice_names.columns.str.replace('unc-47 2','unc-47 batch2')
df_nice_names.columns = df_nice_names.columns.str.replace('unc-47 1','unc-47 batch1')
df_nice_names.columns = df_nice_names.columns.str.replace('-','&#8209;')

# same for cell type names
# df_nice_names['Cell Type']= df_nice_names['Cell Type'].str.replace('_',' ')


# convert df to dict for sending as json to datatables
dict_df = df_nice_names.to_dict(orient='records')
# convert column names into dict for sending as json to datatables
columns = [{"data": item, "title": item} for item in df_nice_names.columns]

#### datatables ####

@tables.route("/", methods=['GET'])
def clientside_table_content():
    return jsonify({'data': dict_df, 'columns': columns})

flask_app.register_blueprint(tables)

@flask_app.route("/")
def clientside_table():
    return render_template("clientside_table.html")


####

@flask_app.route("/test")
def test():
    return render_template("test.html")

# @flask_app.route("/")
# def index():
#     logger.info('Got a request for index!')
#     return render_template("index.html")

@flask_app.route('/submit', methods=['POST', 'GET'])
def receive_submission():
    logger.info('Got a submission!')
    print('PRESSED THE SUBMISSION BUTTOOOON')
    # answer is a dict of json strings containing selected row and column index numbers
    answer = request.form.to_dict(flat=False)
    print(answer)
    print(df.head())

    #first try is in case submission is from table form
    try:
      
        # need to convert the json strings to dict, then to a data frame
        # data1 is the selection for the first group, data2 for the second
        data1 = json.loads(answer['data1'][0])
        data1_df = pd.DataFrame.from_dict(data1[0])

        print(data1_df)
        data2 = json.loads(answer['data2'][0])
        data2_df = pd.DataFrame.from_dict(data2[0])

        # now map the index number to experiment name and cell type name
        group1_df = pd.DataFrame()
        group1_df['cell_type1'] = data1_df['row'].map(df['Cell Type'])
        group1_df['experiment1'] = data1_df['column'].map(pd.Series(df.columns.values))
        # print(group1_df)

        group2_df = pd.DataFrame()
        group2_df['cell_type2'] = data2_df['row'].map(df['Cell Type'])
        group2_df['experiment2'] = data2_df['column'].map(pd.Series(df.columns.values))
        # print(group2_df)

        timestr = time.strftime("%Y%m%d-%H%M%S")
        print(timestr)

        genes = StringIO(json.loads(answer['genes'][0]))
        genes_df = pd.read_csv(genes, names=['selected_genes'])
        print(genes_df)
        print(jobname)
        print('=======')
        jobname_df = pd.DataFrame(columns=['job_name'])
        jobname_df = jobname_df.append({'job_name': 'bla'}, ignore_index=True)

    # if that doesn't work it's a text submission
    except:

        # need to convert the json strings to stringio, then to a data frame
        # data1 is the selection for the first group, data2 for the second

        data1 = StringIO(json.loads(answer['data1'][0]))
        # print(data1)
        group1_df = pd.read_csv(data1, names=['cell_type1', 'experiment1'])
        # print(data1_df)

        data2 = StringIO(json.loads(answer['data2'][0]))
        group2_df = pd.read_csv(data2, names=['cell_type2', 'experiment2'])

        genes = StringIO(json.loads(answer['genes'][0]))
        genes_df = pd.read_csv(genes, names=['selected_genes'])
        print(genes_df)

        # now map the index number to experiment name and cell type name

        timestr = time.strftime("%Y%m%d-%H%M%S")
        print(timestr)

    s3filename = 'submissions/' + email + '%' + timestr + '.csv'

    selected_groups_df = pd.concat([group1_df, group2_df, genes_df, jobname_df], axis=1)
    print(selected_groups_df)

    csv_buffer = StringIO()
    selected_groups_df.to_csv(csv_buffer)
    # files.download("readme.md")


    # return 'derpderp'
    print('RETUN FILE NOW')
    # return send_file('/content/df.csv')
    # return send_file('./foo.txt', as_attachment=True)
    return redirect(url_for('test'))


@flask_app.route('/file_downloads')
def file_downloads():
	try:
		return render_template("""
			<body class="body">
				<div class="container" align="left">
					<a href="/return_files/" target="blank"><button class='btn btn-default'>Download!</button></a>
				</div>
			</body>	
		""")
	except Exception as e:
		return str(e)
  
@flask_app.route('/return_files/')
def return_files():
	try:
		return send_file('./foo.txt', as_attachment=True)#, attachment_filename='python.jpg')
	except Exception as e:
		return str(e)



if __name__ == "__main__":
    flask_app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://bc831ca74783.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [26/Oct/2020 02:00:15] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2020 02:00:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2020 02:00:15] "GET /static/js/clientside_table.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2020 02:00:15] "GET /static/js/clientside_table.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2020 02:00:15] "GET /tables/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2020 02:00:15] "GET /tables/ HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2020 02:00:15] "GET /tables/ HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2020 02:00:15] "GET /tables/ HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2020 02:00:16] "GET /static/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2020 02:00:16] "GET /static/favicon.ico HTTP/1.1" 200 -
[2020-10-26 02:00:26,440] ERROR in app: Exception on /submit [POST]
Traceback (most recent call last):
  File "<ipython-input-92-e3a6700c5d77>", line 98, in receive_submission
    print(jobname)
NameError: na

PRESSED THE SUBMISSION BUTTOOOON
{'data1': ['[[{"row":8,"column":1}]]'], 'data2': ['[[{"row":4,"column":1}]]'], 'genes': ['"nlp\\r\\nflp\\r\\nins"']}
             Cell Type  L2_experiment_1  ...  tph-1_ceh-10  unc-3
0     ABarpaaa_lineage                0  ...             0      0
1                 ADA?                0  ...             1     62
2  ADA_ADE_grandparent                0  ...             0      0
3                  ADE                0  ...             1      2
4          ADE_CEP_PDE              126  ...             0      0

[5 rows x 22 columns]
   row  column
0    8       1
20201026-020026
  selected_genes
0            nlp
1            flp
2            ins


127.0.0.1 - - [26/Oct/2020 02:00:27] "GET /static/js/submission_logic.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [26/Oct/2020 02:00:27] "GET /static/js/submission_logic.js HTTP/1.1" 404 -


In [93]:
!cd scdefg/

In [108]:
# !git -C ./scdefg/ add df.csv static/* templates/*
# !git -C ./scdefg/ status
# !git -C ./scdefg/ commit -m "commiting from colab!"
!git -C ./scdefg/ push -u origin main

# !git -C ./scdefg/ config --global user.email munfred@brandeis.edu
# !git -C ./scdefg/ config --global user.name munfred

fatal: could not read Username for 'https://github.com': No such device or address


In [109]:
!zip -r scdefg.zip scdefg/


  adding: scdefg/ (stored 0%)
  adding: scdefg/LICENSE (deflated 43%)
  adding: scdefg/.gitignore (deflated 46%)
  adding: scdefg/static/ (stored 0%)
  adding: scdefg/static/js/ (stored 0%)
  adding: scdefg/static/js/clientside_table.js (deflated 73%)
  adding: scdefg/static/js/.ipynb_checkpoints/ (stored 0%)
  adding: scdefg/static/favicon.ico (stored 0%)
  adding: scdefg/.git/ (stored 0%)
  adding: scdefg/.git/config (deflated 33%)
  adding: scdefg/.git/HEAD (stored 0%)
  adding: scdefg/.git/refs/ (stored 0%)
  adding: scdefg/.git/refs/remotes/ (stored 0%)
  adding: scdefg/.git/refs/remotes/origin/ (stored 0%)
  adding: scdefg/.git/refs/remotes/origin/HEAD (stored 0%)
  adding: scdefg/.git/refs/heads/ (stored 0%)
  adding: scdefg/.git/refs/heads/main (deflated 2%)
  adding: scdefg/.git/refs/tags/ (stored 0%)
  adding: scdefg/.git/index (deflated 38%)
  adding: scdefg/.git/objects/ (stored 0%)
  adding: scdefg/.git/objects/07/ (stored 0%)
  adding: scdefg/.git/objects/07/011963bf26d9d